In [6]:
# import plotly.io as pio
# pio.renderers.default = "notebook"

In [1]:
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output

app = dash.Dash()
app.layout = html.Div([

    html.H2(id = 'Input H2',
            children=['Input']),
    dcc.Input(id='input',
            value = '글자를 입력하세요'),

    html.Br(),
    html.Br(),

    html.H2(id ='Output H2',
            children=['Output']),
    html.Div(id='div output',
             children=['아직 입력받지 못했습니다.'])
])

@app.callback(Output(component_id='div output',
                     component_property='children'),
            [Input(component_id='input',
                    component_property='value')])
def update_div(value):
    return "입력받은 내용입니다: {}".format(value)


if __name__=='__main__':
    app.run_server()

# 127.0.0.1:8050

In [2]:
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output

options = [
    {'label':'딸기', 'value':3000},
    {'label':'수박', 'value':10000},
    {'label':'바나나', 'value':2000},
    {'label':'키위', 'value':1000},
    {'label':'사과', 'value':1500}
    ]

app = dash.Dash()
app.layout = html.Div([

    html.H3(id='H3 1',
            children=['Label']),
    dcc.Dropdown(id='dropdown',
                 options=options),
    html.Br(),
    html.H3(id='H3 2',
            children=['Value']),
    html.Div(id='div output',
             children=['가격'])
])

@app.callback(Output(component_id='div output',
                     component_property='children'),
            [Input(component_id='dropdown',
                    component_property='value')])
def update_div(value):
    return "{}".format(value)


if __name__=='__main__':
    app.run_server()

# 127.0.0.1:8050

In [1]:
import base64
import io
import os
import sqlite3
from dash import Dash, dcc, html, Input, Output
from dash.exceptions import PreventUpdate
from dash.dependencies import State
from flask import Flask
from moviepy.editor import VideoFileClip

# Flask 애플리케이션 설정
server = Flask(__name__)
app = Dash(__name__, server=server)
app.title = "동영상 재생 시간 저장"

# SQLite DB 초기화
conn = sqlite3.connect('video_database.db')
cursor = conn.cursor()
cursor.execute('''
    CREATE TABLE IF NOT EXISTS videos (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        filename TEXT,
        duration INTEGER
    )
''')
conn.commit()

# Dash 애플리케이션 레이아웃
app.layout = html.Div([
    dcc.Upload(
        id='upload-video',
        children=html.Div([
            '동영상 파일을 드래그 앤 드롭하거나 ',
            html.A('파일을 선택하세요.')
        ]),
        multiple=False
    ),
    html.Div(id='video-duration-output'),
])

# 파일 업로드 콜백
@app.callback(
    Output('video-duration-output', 'children'),
    Input('upload-video', 'contents'),
    prevent_initial_call=True
)
def update_output(contents):
    if contents is None:
        raise PreventUpdate

    content_type, content_string = contents.split(',')
    decoded = base64.b64decode(content_string)
    video_duration = get_video_duration(decoded)

    # DB에 저장
    filename = 'uploaded_video.mp4'  # 파일명은 적절히 수정하세요
    store_video_duration(filename, video_duration)

    return f'동영상 재생 시간: {video_duration} 초'

# 동영상 재생 시간 추출 함수
def get_video_duration(video_content):
    try:
        video_file = io.BytesIO(video_content)
        with VideoFileClip(video_file) as video:
            return video.duration
    except Exception as e:
        print(f"동영상 재생 시간 추출 중 오류 발생: {e}")
        return None

# 동영상 재생 시간을 DB에 저장하는 함수
def store_video_duration(filename, duration):
    conn = sqlite3.connect('video_database.db')
    cursor = conn.cursor()
    cursor.execute('INSERT INTO videos (filename, duration) VALUES (?, ?)', (filename, duration))
    conn.commit()
    conn.close()

# 애플리케이션 실행
if __name__ == '__main__':
    app.run_server(debug=True)


# 127.0.0.1:8050
# 문장을 클릭해서 동영상 파일 업로드

In [4]:
import base64
import io
import sqlite3
from dash import Dash, dcc, html, Input, Output
from dash.exceptions import PreventUpdate
from flask import Flask
from moviepy.editor import VideoFileClip

# Flask 애플리케이션 설정
server = Flask(__name__)
app = Dash(__name__, server=server)
app.title = "동영상 재생 시간 저장"

# SQLite DB 초기화
conn = sqlite3.connect('video_database.db')
cursor = conn.cursor()
cursor.execute('''
    CREATE TABLE IF NOT EXISTS videos (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        filename TEXT,
        duration INTEGER
    )
''')
conn.commit()

# Dash 애플리케이션 레이아웃
app.layout = html.Div([
    dcc.Upload(
        id='upload-video',
        children=html.Div([
            '동영상 파일을 드래그 앤 드롭하거나 ',
            html.A('파일을 선택하세요.')
        ]),
        multiple=False
    ),
    html.Div(id='video-duration-output'),
])

# 파일 업로드 콜백
@app.callback(
    Output('video-duration-output', 'children'),
    Input('upload-video', 'contents'),
    prevent_initial_call=True
)
def update_output(contents):
    if contents is None:
        raise PreventUpdate

    content_type, content_string = contents.split(',')
    decoded = base64.b64decode(content_string)
    video_duration = get_video_duration(decoded)

    # DB에 저장
    filename = 'uploaded_video.mp4'  # 파일명은 적절히 수정하세요
    store_video_duration(filename, video_duration)

    return f'동영상 재생 시간: {video_duration} 초'

# 동영상 재생 시간 추출 함수
def get_video_duration(video_content):
    try:
        # 파일로 저장
        video_file_path = 'temp_video.mp4'
        with open(video_file_path, 'wb') as video_file:
            video_file.write(video_content)
        
        # VideoFileClip에 파일 경로를 직접 제공
        video_clip = VideoFileClip(video_file_path)
        duration = int(video_clip.duration)
        video_clip.close()

        # 임시 파일 삭제
        os.remove(video_file_path)
        
        return duration
    except Exception as e:
        print(f"동영상 재생 시간 추출 중 오류 발생: {e}")
        return None

# 동영상 재생 시간을 DB에 저장하는 함수
def store_video_duration(filename, duration):
    conn = sqlite3.connect('video_database.db')
    cursor = conn.cursor()
    cursor.execute('INSERT INTO videos (filename, duration) VALUES (?, ?)', (filename, duration))
    conn.commit()
    conn.close()

# 애플리케이션 실행
if __name__ == '__main__':
    app.run_server(debug=True)


# 127.0.0.1:8050